In [1]:
import random as rand
import numpy as np 
import cv2
import glob
import time
import os

import numpy as np

from skimage import data, draw, io, img_as_ubyte
from skimage.feature import hog
import joblib

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.transform import rescale, resize, downscale_local_mean
import numpy as np 

from features import FeaturesRGB
import matplotlib.pyplot as plt

### Classifier that stores the model

In [2]:
class BinaryClassifier:

    def __init__(self, svc, scaler):
        self.svc = svc
        self.scaler = scaler 

    def predict(self, f):
        f = self.scaler.transform([f])
        r = self.svc.predict(f)
        return int(r[0]) 

In [3]:
class Slider:
    
    def __init__(self, frame, classifier, hog_params):
        self.frame = frame
        self.classifier = classifier
        self.hog_params = hog_params
        self.bd_box = hog_params['bounding_box_size']

    def stripping(self, pad, start_pos_x, start_pos_y):

        self.pad = pad
        self.start_pos_x = start_pos_x
        self.start_pos_y = start_pos_y
        
        pad_h =  self.bd_box[0] + pad * 2
        pad_w =  self.bd_box[1] + pad * 2
        
        #crop the LFT area
        strip = self.frame[start_pos_y :self.frame.shape[0] - start_pos_y,\
                                start_pos_x: self.frame.shape[1] - start_pos_x, :]
        
        #resize 
        self.scale_factor = self.frame.shape[0] / pad_h
        self.strip_resized = resize(strip, (int(strip.shape[0] / self.scale_factor), \
                                            int(strip.shape[1] / self.scale_factor)), \
                                            anti_aliasing=True)
        
        


    def sliding(self, increment):
        
        box = []
        
        ftrRGB = FeaturesRGB(self.hog_params)
        
        x_win_cnt = ( (self.strip_resized.shape[1] - self.bd_box[1]) // increment )
        y_win_cnt = ( (self.strip_resized.shape[0] - self.bd_box[0]) // increment )
        
        #print("number of window: ", x_win_cnt, y_win_cnt)
        
        
        for x_i in range(0, x_win_cnt):
            for y_i in range(0, y_win_cnt):
                strip_crop = self.strip_resized[y_i * increment : y_i * increment + self.bd_box[0],\
                                                x_i * increment : x_i * increment + self.bd_box[1], :]
                #print("strip_crop: ", strip_crop.shape)
                
                hog_flatten = ftrRGB.cal_hog(strip_crop, 0)
                
                cls_rslt = self.classifier.predict(hog_flatten)
                #print("result: ", cls_rslt)
                
                if cls_rslt:
                    tmp_y = (y_i * increment) * self.scale_factor + self.start_pos_y
                    tmp_x = (x_i * increment) * self.scale_factor + self.start_pos_x
                    
                    tmp_y_end = (y_i * increment + self.bd_box[0]) * self.scale_factor + self.start_pos_y
                    tmp_x_end = (x_i * increment + self.bd_box[1]) * self.scale_factor + self.start_pos_x
                    
                    
                    box.append((int(tmp_y), int(tmp_x), \
                                int(tmp_y_end), int(tmp_x_end)))
                    
                    
                    #fig, ax = plt.subplots(1,1, figsize=(10, 5))
                    #ax.title.set_text('strip_crop')
                    #ax.imshow(strip_crop);
        
        return box
            


## Main program

In [4]:
svc = joblib.load('../svc_validated.pkl')
scaler = joblib.load('../scaler_validated.pkl')


hog_params = {
    'bounding_box_size': [384, 128],   
    'orientations': 11, # number of bins
    'pixels_per_cell': (8, 8), # normally (8,8), refers to 8x8 pixels are used to calculate hog
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}


cls = BinaryClassifier(svc, scaler)
ftrRGB = FeaturesRGB(hog_params)


### Getting files

In [23]:
lft1_posit_path = glob.glob('../../mlft_classify/originals/negative/hcng/unchanged/*.jpg')

lft1crop_posit_path = '../../mlft_classify/originals/negative/hcng/cropped/'
lft1nochg_posit_path = '../../mlft_classify/originals/negative/hcng/unchanged/'


In [24]:
slider_inc = 8

for path in lft1_posit_path: 
    # Get file name
    f_split = os.path.basename(path)
    name_tmp = os.path.splitext(f_split)[0]
    
    lft_img = io.imread(path)
    
    #print(type(name_tmp), type(lft1nochg_posit_path))
    
    slider = Slider(lft_img, classifier = cls,  hog_params = hog_params)
    slider.stripping(64, lft_img.shape[1] // 4, 32)
    
    
    print("Sliding: \"", name_tmp, "\"...", end='')
    box = slider.sliding(slider_inc)
    
    print("Finished, bounding box found? ", (len(box) > 0))
    
    # crop image based on the bounding box
    if len(box) > 0:
        start_yx = box[0]
        end_yx = box[-1]
        
        
        v_pad = (start_yx[0] - end_yx[2]) // 4
        h_pad = (start_yx[1] - end_yx[3]) // 6

        strip = lft_img[start_yx[0] - v_pad : end_yx[2] + v_pad,\
                                    start_yx[1] - h_pad : end_yx[3] + h_pad, :]
        
        #strip1 = lft_img[start_yx[0]  : end_yx[2] ,\
                                    #start_yx[1]  : end_yx[3], :]
            

            
        io.imsave(lft1crop_posit_path  + name_tmp + "_cropped.jpg", img_as_ubyte(strip))
        
        #fig, ax = plt.subplots(1,1, figsize=(20, 10))
        #ax.imshow(strip)
        #break
    
    else:
        io.imsave(lft1nochg_posit_path  + name_tmp + ".jpg", img_as_ubyte(lft_img))

        
    

Sliding: " 12 "...Finished, bounding box found?  False
Sliding: " 19 "...Finished, bounding box found?  True
Sliding: " 20 "...Finished, bounding box found?  False
